# Проектная работа
# Исследование книжного рынка

# Описание исследования
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. <br>
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. <br>
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Цель исследования
Проанализировать базу данных для формулирования ценностного предложение для нового продукта.

# Задания

1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

# Описание данных
**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Оглавление

1. [Импорт библиотек и установка параметров](#start) <br> <br>
2. [Исследование таблиц](#data_analysis)
    * [Таблица `books`](#books)
    * [Таблица `authors`](#authors)
    * [Таблица `publishers`](#publishers)
    * [Таблица `ratings`](#ratings)
    * [Таблица `reviews`](#reviews) <br> <br>
3. [Задания](#tasks)
    * [Задание 1](#task1)
    * [Задание 2](#task2)
    * [Задание 3](#task3)
    * [Задание 4](#task4)
    * [Задание 5](#task5) <br> <br>
4. [Итоговый вывод](#final_conclusion)

<a id="start"></a> 
## Импорт библиотек и установка параметров

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

<a id="data_analysis"></a> 
## Исследование таблиц

<a id="books"></a> 
### Таблица `books`

In [3]:
query = '''SELECT * FROM books LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''
           SELECT *
           FROM books
           WHERE 
                 book_id IS NULL OR
                 author_id IS NULL OR
                 title IS NULL OR
                 num_pages IS NULL OR
                 num_pages = 0 OR
                 publication_date IS NULL OR
                 publisher_id IS NULL
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,book_id,author_id,title,num_pages,publication_date,publisher_id


Пропуски в данных отсутствуют.

In [5]:
query = '''
           SELECT count(book_id),
                  count(DISTINCT book_id) 
           FROM books
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,count,count
0,1000,1000


В таблице `books` хранится информация о 1000 книг.

In [6]:
query = '''
           SELECT MIN(publication_date),
                  MAX(publication_date) 
           FROM books
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,min,max
0,1952-12-01,2020-03-31


Минимальная дата публикации книги: 1 января 1952. <br>
Максимальная дата публикации книги: 31 марта 2020.

<a id="authors"></a> 
### Таблица `authors`

In [7]:
query = '''SELECT * FROM authors LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
query = '''
           SELECT *
           FROM authors
           WHERE 
                 author_id IS NULL OR
                 author IS NULL
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,author_id,author


Пропуски в данных отсутствуют.

In [9]:
query = '''
           SELECT count(author_id),
                  count(DISTINCT author_id) 
           FROM authors
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,count,count
0,636,636


В таблице `authors` хранится информация о 636 авторах.

<a id="publishers"></a> 
### Таблица `publishers`

In [10]:
query = '''SELECT * FROM publishers LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [11]:
query = '''
           SELECT *
           FROM publishers
           WHERE 
                 publisher_id IS NULL OR
                 publisher IS NULL
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,publisher_id,publisher


Пропуски в данных отсутствуют.

In [12]:
query = '''
           SELECT count(publisher_id),
                  count(DISTINCT publisher_id) 
           FROM publishers
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,count,count
0,340,340


В таблице `publishers` хранится информация о 340 издательствах.

<a id="ratings"></a> 
### Таблица `ratings`

In [13]:
query = '''SELECT * FROM ratings LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [14]:
query = '''
           SELECT *
           FROM ratings
           WHERE 
                 rating_id IS NULL OR
                 book_id IS NULL OR
                 username IS NULL OR
                 rating IS NULL
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,rating_id,book_id,username,rating


Пропуски в данных отсутствуют.

In [15]:
query = '''
           SELECT *
           FROM ratings
           WHERE 
                 rating < 1 OR
                 rating > 5
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,rating_id,book_id,username,rating


В данных по рейтингу аномалии не обнаружено.

In [16]:
query = '''
           SELECT count(*)
           FROM ratings
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,count
0,6456


В таблице `ratings` хранится информация о 6456 оценках пользователей.

<a id="reviews"></a> 
### Таблица `reviews`

In [17]:
query = '''SELECT * FROM reviews LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [18]:
query = '''
           SELECT *
           FROM reviews
           WHERE 
                 review_id IS NULL OR
                 book_id IS NULL OR
                 username IS NULL OR
                 text IS NULL
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,review_id,book_id,username,text


Пропуски в данных отсутствуют.

In [19]:
query = '''
           SELECT count(*)
           FROM reviews
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,count
0,2793


В таблице `reviews` хранится информация о 2793 обзорах пользователей.

<a id="tasks"></a> 
## Задания
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Сделайте по одному SQL-запросу для решения каждого задания...

<a id="task1"></a> 
### Задание 1
**Посчитайте, сколько книг вышло после 1 января 2000 года.**

In [20]:
query = '''
           SELECT COUNT(*)
           FROM books
           WHERE publication_date :: date > '2000-01-01'
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,count
0,819


**Вывод** <br>
После ***1 января 2000 года*** вышло ***819*** книг. <br>
Всего в таблице `books` содержится ***1000*** книг.

<a id="task2"></a> 
### Задание 2
**Для каждой книги посчитайте количество обзоров и среднюю оценку.**

In [21]:
query = '''
           WITH t_rev AS(
           SELECT b.book_id,
                  b.title AS title,
                  a.author,
                  COUNT (b.book_id) AS count_reviews
           FROM books AS b
           LEFT JOIN reviews AS r
           ON b.book_id = r.book_id
           JOIN authors AS a
           ON b.author_id = a.author_id
           GROUP BY b.book_id, a.author
           ORDER BY count_reviews DESC),
           
           t_rat AS(
           SELECT DISTINCT book_id,
                  ROUND(AVG(rating) OVER (PARTITION BY book_id), 2) AS avg_rating
           FROM ratings)
           
           SELECT t_rev.book_id,
                  t_rev.title,
                  t_rev.author,
                  t_rev.count_reviews,
                  t_rat.avg_rating
           FROM t_rev
           JOIN t_rat
           ON t_rev.book_id = t_rat.book_id
           ORDER BY t_rev.count_reviews DESC,
                    t_rat.avg_rating DESC
           LIMIT 20
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,book_id,title,author,count_reviews,avg_rating
0,948,Twilight (Twilight #1),Stephenie Meyer,7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,6,4.29
3,656,The Book Thief,Markus Zusak/Cao Xuân Việt Khương,6,4.26
4,734,The Glass Castle,Jeannette Walls,6,4.21
5,497,Outlander (Outlander #1),Diana Gabaldon,6,4.13
6,750,The Hobbit or There and Back Again,J.R.R. Tolkien,6,4.13
7,695,The Curious Incident of the Dog in the Night-Time,Mark Haddon,6,4.08
8,779,The Lightning Thief (Percy Jackson and the Oly...,Rick Riordan,6,4.08
9,963,Water for Elephants,Sara Gruen,6,3.98


**Вывод**

* Больше всего обзоров (***7***) на книгу ***Stephenie Meyer "Twilight (Twilight #1)"***. Средний рейтинг книги - ***3.66***. <br>
* Среди книг с ***6-ю*** обзорами самый высокий рейтинг (***4.41***) у книги ***J.K. Rowling "Harry Potter and the Prisoner of Azkaban"***. <br>
На втором месте среди книг с 6 обзорами с рейтингом ***4.29*** ещё одна книга ***J.K. Rowling "Harry Potter and the Chamber of Secrets"***. <br>
Замыкает тройку книга ***Markus Zusak "The Book Thief"*** с рейтингом ***4.26***.
* Из книг с ***5-ю*** обзорами лучший рейтинг у книги ***Lisa See "Snow Flower and the Secret Fan"*** и составляет ***4.64***.

<a id="task3"></a> 
### Задание 3
**Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.**

In [22]:
query = '''      
           SELECT p.publisher_id AS id,
                  p.publisher,
                  COUNT(p.publisher_id) AS count_books
           FROM publishers AS p
           JOIN books AS b
           ON b.publisher_id = p.publisher_id
           WHERE num_pages > 50
           GROUP BY p.publisher_id
           ORDER BY count_books DESC
           LIMIT 1
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,id,publisher,count_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19


**Вывод**

Издательство ***Penguin Books*** выпустило ***42*** книги толще 50 страниц и это наибольший показатель.

<a id="task4"></a> 
### Задание 4
**Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.**

In [23]:
query = '''
           WITH t1 AS (
           SELECT DISTINCT book_id,
                  COUNT(rating) OVER (PARTITION BY book_id) AS count_ratings,
                  AVG(rating) OVER (PARTITION BY book_id) AS avg_ratings
           FROM ratings
           ORDER BY count_ratings DESC),
           
           t2 AS (
           SELECT t1.book_id,
                  t1.avg_ratings,
                  a.author_id,
                  a.author
           FROM t1
           JOIN books AS b
           ON t1.book_id = b.book_id
           JOIN authors AS a
           ON b.author_id = a.author_id
           WHERE t1.count_ratings >= 50)
           
           SELECT DISTINCT author_id,
                  author,
                  COUNT(author_id) OVER (PARTITION BY author_id) AS count_books,
                  ROUND(AVG(avg_ratings) OVER (PARTITION BY author_id), 3) AS avg_rating
           FROM t2
           ORDER BY avg_rating DESC
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,author_id,author,count_books,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4,4.284
1,402,Markus Zusak/Cao Xuân Việt Khương,1,4.264
2,240,J.R.R. Tolkien,2,4.258
3,376,Louisa May Alcott,1,4.192
4,498,Rick Riordan,1,4.081
5,621,William Golding,1,3.901
6,235,J.D. Salinger,1,3.826
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,1,3.789
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,1,3.788
9,106,Dan Brown,2,3.755


**Вывод**

Автором с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, является ***J.K. Rowling***. Учтено ***4*** книги. Средний рейтинг - ***4.284***. <br>
На 2ом месте ***Markus Zusak*** с ***1*** книгой и рейтингом ***4.264***. <br>
Замыкает тройку лидеров ***J.R.R. Tolkien*** с ***2*** книгами и рейтингом ***4.258***. <br>
Всего 14 подходит под условия задачи.

<a id="task5"></a> 
### Задание 5
**Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

In [24]:
query = '''
           WITH t_all_users AS (
           SELECT DISTINCT username,
                  COUNT(rating) OVER (PARTITION BY username) AS count_ratings
           FROM ratings),
    
           t_good_users AS (
           SELECT DISTINCT username,
                  COUNT(review_id) OVER (PARTITION BY username) AS count_reviews
           FROM reviews
           WHERE username IN (SELECT username
                              FROM t_all_users
                              WHERE count_ratings > 48)
           ORDER BY count_reviews DESC)
                              
           SELECT AVG(count_reviews)
           FROM t_good_users
        '''

pd.io.sql.read_sql(sql=text(query), con=engine.connect())

,avg
0,24.0


**Вывод**

Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет ***24***.

<a id="final_conclusion"></a> 
## Итоговый вывод



* В таблице books хранится информация о 1000 книг.
* После 1 января 2000 года вышло 819 книг.
* Минимальная дата публикации книги: 1 января 1952.
* Максимальная дата публикации книги: 31 марта 2020.


* В таблице authors хранится информация о 636 авторах.
* Автором с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, является J.K. Rowling. Учтено 4 книги. Средний рейтинг - 4.284. На 2ом месте Markus Zusak с 1 книгой и рейтингом 4.264. Замыкает тройку лидеров J.R.R. Tolkien с 2 книгами и рейтингом 4.258.


* В таблице publishers хранится информация о 340 издательствах.
* Издательство ***Penguin Books*** выпустило ***42*** книги толще 50 страниц и это наибольший показатель


* В таблице ratings хранится информация о 6456 оценках пользователей.


* В таблице reviews хранится информация о 2793 обзорах пользователей.
* Больше всего обзоров (7) на книгу Stephenie Meyer "Twilight (Twilight #1)". Средний рейтинг книги - 3.66. Среди книг с 6-ю обзорами самый высокий рейтинг (4.41) у книги J.K. Rowling "Harry Potter and the Prisoner of Azkaban". На втором месте среди книг с 6 обзорами с рейтингом 4.29 ещё одна книга J.K. Rowling "Harry Potter and the Chamber of Secrets". Замыкает тройку книга Markus Zusak "The Book Thief" с рейтингом 4.26. Из книг с 5-ю обзорами лучший рейтинг у книги Lisa See "Snow Flower and the Secret Fan" и составляет 4.64.
* Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24.